In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('journal/strat_fitx.csv')

In [3]:
fitx = pd.read_csv('data/fitx.txt')

In [4]:
new_fitx = fitx[1015:].copy()
new_fitx.index = range(new_fitx.shape[0])

In [5]:
consolidated = df.merge(new_fitx[['Date','Time']], left_index=True, right_index=True)

In [6]:
consolidated = consolidated[['Date','Time','mid_price','Position','PnL']]

In [7]:
consolidated.columns = ['Date', 'Time', 'price', 'side', 'profit']

In [8]:
consolidated['contracts'] = np.zeros(consolidated.shape[0])

In [9]:
consolidated['contracts'].iloc[0] = 1

/home/zekechyo/anaconda3/envs/stockExchange/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
consolidated['last_side']=consolidated['side'].shift(1)

In [11]:
consolidated['last_side'].iloc[0]=0

In [12]:
contracts = consolidated[['side','last_side']]

In [13]:
contracts['contracts'] = contracts['side']!=contracts['last_side']

/home/zekechyo/anaconda3/envs/stockExchange/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
contracts['contracts'] = contracts['contracts'].astype(int)

/home/zekechyo/anaconda3/envs/stockExchange/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
contracts['contracts'] = contracts['contracts']*2

/home/zekechyo/anaconda3/envs/stockExchange/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
consolidated['contracts'].iloc[1:] = contracts['contracts'][1:]

In [17]:
consolidated.drop(columns=['last_side'], inplace=True)

In [18]:
consolidated['commodity'] = ['TXF']*consolidated.shape[0]

In [19]:
consolidated['strategy'] = ['AI_Sim']*consolidated.shape[0]

In [20]:
consolidated['action'] = np.zeros(consolidated.shape[0])

In [21]:
consolidated['action'] = consolidated['side']*consolidated['contracts']

In [22]:
trans_dict = {'-1.0':'Entry_Short', '1.0':'Entry_Long', '-2.0':'Entry_Short', '2.0':'Entry_Long', '0.0':'Hold', '-0.0':'Hold'}

In [23]:
consolidated['action']=consolidated['action'].apply(lambda x: trans_dict[str(x)])

In [24]:
consolidated[consolidated['action']=='Hold']=None

In [25]:
consolidated.dropna(inplace=True)

In [26]:
consolidated.index = range(consolidated.shape[0])

In [27]:
consolidated['last_action'] = consolidated['action'].shift(1)

In [28]:
consolidated.Time = pd.to_datetime(consolidated.Time, format='%H:%M:%S')

In [29]:
consolidated2 = consolidated.copy()

In [30]:
consolidated2.index = [ 2*i for i in range(consolidated2.shape[0])]

In [31]:
consolidated3 = consolidated2.copy()

In [32]:
consolidated3.index=[2*i+1 for i in range(consolidated3.shape[0])]

In [33]:
consolidated3[:-1].price = np.asarray(consolidated2[1:].price)

/home/zekechyo/anaconda3/envs/stockExchange/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [34]:
trans_dict = {'Entry_Short':'Exit_Short', 'Entry_Long':'Exit_Long'}
consolidated3['action'] = consolidated3.action.apply(lambda x: trans_dict[x])

In [35]:
import datetime

In [36]:
datetime.timedelta(seconds=15)

datetime.timedelta(0, 15)

In [37]:
consolidated3.Time += datetime.timedelta(seconds=15)

In [38]:
consolidated3.side = -1*consolidated3.side

In [39]:
consolidated2.drop(columns=['last_action'], inplace=True)

In [40]:
consolidated3.drop(columns=['last_action'], inplace=True)

In [41]:
consolidated2

,Date,Time,price,side,profit,contracts,commodity,strategy,action
0,2009/1/8,1900-01-01 10:41:00,4556.0,-1.0,0.5,1.0,TXF,AI_Sim,Entry_Short
2,2009/1/8,1900-01-01 10:42:00,4555.5,1.0,-1.5,2.0,TXF,AI_Sim,Entry_Long
4,2009/1/8,1900-01-01 10:43:00,4554.0,-1.0,-0.5,2.0,TXF,AI_Sim,Entry_Short
6,2009/1/8,1900-01-01 10:44:00,4554.5,1.0,1.0,2.0,TXF,AI_Sim,Entry_Long
8,2009/1/8,1900-01-01 10:46:00,4553.0,-1.0,-0.0,2.0,TXF,AI_Sim,Entry_Short
...,...,...,...,...,...,...,...,...,...
631414,2019/12/13,1900-01-01 09:46:00,11948.5,1.0,5.5,2.0,TXF,AI_Sim,Entry_Long
631416,2019/12/13,1900-01-01 09:52:00,11966.5,-1.0,7.0,2.0,TXF,AI_Sim,Entry_Short
631418,2019/12/13,1900-01-01 10:00:00,11949.5,1.0,3.5,2.0,TXF,AI_Sim,Entry_Long
631420,2019/12/13,1900-01-01 10:03:00,11951.5,-1.0,5.0,2.0,TXF,AI_Sim,Entry_Short


In [42]:
oldIndex_2 = np.asarray(consolidated2.index)

In [43]:
oldIndex_3 = np.asarray(consolidated3.index)

In [44]:
oldIndex_2

array([     0,      2,      4, ..., 631418, 631420, 631422])

In [45]:
oldIndex_3

array([     1,      3,      5, ..., 631419, 631421, 631423])

In [46]:
consolidated2.index = range(consolidated2.shape[0])
consolidated3.index = range(consolidated3.shape[0])

In [47]:
save_2 = pd.DataFrame(consolidated2.iloc[0].to_dict(), index=[0], columns=consolidated2.columns)
save_3 = pd.DataFrame(consolidated3.iloc[-1].to_dict(), index=[consolidated3.shape[0]-1], columns=consolidated3.columns)
consolidated2.drop([0], inplace=True)
consolidated3.drop([consolidated.shape[0]-1], inplace=True)

In [48]:
consolidated2.index = range(consolidated2.shape[0])
consolidated3.index = range(consolidated3.shape[0])

In [49]:
consolidated3['price'] = consolidated2['price']

In [50]:
consolidated2 = pd.concat([save_2, consolidated2], axis=0, ignore_index=True)
consolidated3 = pd.concat([consolidated3, save_3, ], axis=0, ignore_index=True)

In [51]:
save_3

,Date,Time,price,side,profit,contracts,commodity,strategy,action
315711,2019/12/13,1900-01-01 10:16:15,11930.0,-1.0,2.5,2.0,TXF,AI_Sim,Exit_Long


In [52]:
save_2

,Date,Time,price,side,profit,contracts,commodity,strategy,action
0,2009/1/8,1900-01-01 10:41:00,4556.0,-1.0,0.5,1.0,TXF,AI_Sim,Entry_Short


In [53]:
assert consolidated2.shape[0] == len(oldIndex_2)
assert consolidated3.shape[0] == len(oldIndex_3)

In [54]:
consolidated2.index=oldIndex_2
consolidated3.index=oldIndex_3

In [55]:
consolidated2

,Date,Time,price,side,profit,contracts,commodity,strategy,action
0,2009/1/8,1900-01-01 10:41:00,4556.0,-1.0,0.5,1.0,TXF,AI_Sim,Entry_Short
2,2009/1/8,1900-01-01 10:42:00,4555.5,1.0,-1.5,2.0,TXF,AI_Sim,Entry_Long
4,2009/1/8,1900-01-01 10:43:00,4554.0,-1.0,-0.5,2.0,TXF,AI_Sim,Entry_Short
6,2009/1/8,1900-01-01 10:44:00,4554.5,1.0,1.0,2.0,TXF,AI_Sim,Entry_Long
8,2009/1/8,1900-01-01 10:46:00,4553.0,-1.0,-0.0,2.0,TXF,AI_Sim,Entry_Short
...,...,...,...,...,...,...,...,...,...
631414,2019/12/13,1900-01-01 09:46:00,11948.5,1.0,5.5,2.0,TXF,AI_Sim,Entry_Long
631416,2019/12/13,1900-01-01 09:52:00,11966.5,-1.0,7.0,2.0,TXF,AI_Sim,Entry_Short
631418,2019/12/13,1900-01-01 10:00:00,11949.5,1.0,3.5,2.0,TXF,AI_Sim,Entry_Long
631420,2019/12/13,1900-01-01 10:03:00,11951.5,-1.0,5.0,2.0,TXF,AI_Sim,Entry_Short


In [56]:
consolidated3

,Date,Time,price,side,profit,contracts,commodity,strategy,action
1,2009/1/8,1900-01-01 10:41:15,4555.5,1.0,0.5,1.0,TXF,AI_Sim,Exit_Short
3,2009/1/8,1900-01-01 10:42:15,4554.0,-1.0,-1.5,2.0,TXF,AI_Sim,Exit_Long
5,2009/1/8,1900-01-01 10:43:15,4554.5,1.0,-0.5,2.0,TXF,AI_Sim,Exit_Short
7,2009/1/8,1900-01-01 10:44:15,4553.0,-1.0,1.0,2.0,TXF,AI_Sim,Exit_Long
9,2009/1/8,1900-01-01 10:46:15,4554.5,1.0,-0.0,2.0,TXF,AI_Sim,Exit_Short
...,...,...,...,...,...,...,...,...,...
631415,2019/12/13,1900-01-01 09:46:15,11966.5,-1.0,5.5,2.0,TXF,AI_Sim,Exit_Long
631417,2019/12/13,1900-01-01 09:52:15,11949.5,1.0,7.0,2.0,TXF,AI_Sim,Exit_Short
631419,2019/12/13,1900-01-01 10:00:15,11951.5,-1.0,3.5,2.0,TXF,AI_Sim,Exit_Long
631421,2019/12/13,1900-01-01 10:03:15,11930.0,1.0,5.0,2.0,TXF,AI_Sim,Exit_Short


In [57]:
consolidated = pd.concat([consolidated3, consolidated2]).sort_index()

In [58]:
consolidated['contracts'] = np.ones(consolidated.shape[0])

In [59]:
consolidated.columns=['date','time','price','side','profit','contracts','commodity','strategy','action']

In [60]:
consolidated = consolidated[['date','time','commodity','action','side','contracts','price','strategy']]

In [61]:
consolidated['time']=consolidated['time'].apply(lambda x: x.strftime('%H:%M:%S'))

In [62]:
consolidated.to_csv('AISim.csv', index=False)

In [63]:
consolidated

,date,time,commodity,action,side,contracts,price,strategy
0,2009/1/8,10:41:00,TXF,Entry_Short,-1.0,1.0,4556.0,AI_Sim
1,2009/1/8,10:41:15,TXF,Exit_Short,1.0,1.0,4555.5,AI_Sim
2,2009/1/8,10:42:00,TXF,Entry_Long,1.0,1.0,4555.5,AI_Sim
3,2009/1/8,10:42:15,TXF,Exit_Long,-1.0,1.0,4554.0,AI_Sim
4,2009/1/8,10:43:00,TXF,Entry_Short,-1.0,1.0,4554.0,AI_Sim
...,...,...,...,...,...,...,...,...
631419,2019/12/13,10:00:15,TXF,Exit_Long,-1.0,1.0,11951.5,AI_Sim
631420,2019/12/13,10:03:00,TXF,Entry_Short,-1.0,1.0,11951.5,AI_Sim
631421,2019/12/13,10:03:15,TXF,Exit_Short,1.0,1.0,11930.0,AI_Sim
631422,2019/12/13,10:16:00,TXF,Entry_Long,1.0,1.0,11930.0,AI_Sim
